In [1]:
import pandas as pd

#Load the web_events.csv data set into a Pandas dataframe. 

In [2]:
events = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/web_events.csv')

events.info

events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


# Convert the values in the timestamp field to datetimes. 

In [19]:
events['timestamp'] = pd.to_datetime(events['timestamp'], unit='ms')
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   timestamp      datetime64[ns]
 1   visitorid      int64         
 2   event          object        
 3   itemid         int64         
 4   transactionid  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 105.1+ MB


Extract different time units from the timestamp field. Aggregate on each one, counting the number of records, and see what insights you can discover for each type of event.

In [31]:
events['day'] = events['timestamp'].dt.day
events['month'] = events['timestamp'].dt.month
events['year'] = events['timestamp'].dt.year

In [32]:
day_of_month = events.groupby('day').count()

day_of_month['timestamp'].sort_values(ascending=False)

day
14    103072
7     102839
15    102451
10    100289
8      99072
13     98174
4      98103
11     97610
6      97340
3      97334
9      96252
17     95948
12     94946
5      94349
26     93553
16     91599
25     90921
18     89176
27     85541
19     84312
20     82351
22     80932
28     80799
29     80723
24     80703
21     78672
2      77508
1      77197
23     75639
30     74730
31     53966
Name: timestamp, dtype: int64

We have more records for the middle of the month than any other time period. We see that the 14th and 15th have the most and 3rd most records respectively. 

We have the fewest records for the last two days of the month (30 and 31). It isn't suprising that we have fewer records for the 31st since not all months have 31 days. 

In [33]:
month = events.groupby('month').count()

month['timestamp'].sort_values(ascending=False)

month
7    697984
6    610393
5    590652
8    553362
9    303710
Name: timestamp, dtype: int64

We only have records from May to September. July has the most records of any month, with September containing the fewest records. 

In [34]:
year = events.groupby('year').count()

year['timestamp'].sort_values(ascending=False)

year
2015    2756101
Name: timestamp, dtype: int64

We only have data from the year 2015.

# Round datetimes by hour, aggregate, and see what insights you can discover.

In [39]:
events['hour'] = events['timestamp'].dt.round('H')
events['hour'] = events['hour'].dt.hour

hour = events.groupby('hour').count()

hour['timestamp'].sort_values(ascending=False)

hour
21    186821
20    186163
18    182608
19    181444
22    180913
17    172255
23    168257
4     151716
0     151210
3     148576
16    147325
2     142286
1     141481
5     135492
15    104702
6      99396
14     63954
7      58369
13     40588
8      33111
12     24735
9      20755
11     17686
10     16258
Name: timestamp, dtype: int64

Most of our web events occur between 5-11 PM and our fewest events occur between 6AM-2PM. So most of our events occur in the evening hours when people are home from work. Not suprisingly the fewest events happen when people are either at work or asleep. 

#Load the life_expectancy.csv data set into a Pandas dataframe. 

In [3]:
life_df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/life_expectancy.csv')

life_df.head()

life_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,65.662,66.074,66.444,66.787,67.113,67.435,67.762,68.095,68.436,68.784,69.140,69.498,69.851,70.191,70.519,70.833,71.140,71.441,71.736,72.023,72.293,72.538,72.751,72.929,73.071,73.181,73.262,73.325,73.378,73.425,73.468,73.509,73.544,73.573,73.598,73.622,73.646,73.671,73.700,73.738,73.787,73.853,73.937,74.038,74.156,74.287,74.429,74.576,74.725,74.872,75.016,75.158,75.299,75.440,75.582,75.725,75.867
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.292,32.742,33.185,33.624,34.060,34.495,34.928,35.361,35.796,36.234,36.678,37.128,37.587,38.056,38.540,39.039,39.556,40.092,40.650,41.234,41.853,42.513,43.217,43.963,44.747,45.566,46.417,47.288,48.164,49.028,49.856,50.627,51.331,51.968,52.539,53.055,53.533,53.997,54.468,54.959,55.482,56.044,56.637,57.250,57.875,58.500,59.110,59.694,60.243,60.754,61.226,61.666,62.086,62.494,62.895,63.288,63.673
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,33.251,33.573,33.914,34.272,34.645,35.031,35.426,35.828,36.234,36.640,37.047,37.460,37.878,38.297,38.712,39.110,39.478,39.810,40.099,40.344,40.546,40.710,40.848,40.970,41.085,41.193,41.292,41.382,41.471,41.572,41.696,41.855,42.060,42.329,42.677,43.125,43.695,44.385,45.192,46.105,47.113,48.200,49.341,50.508,51.676,52.833,53.974,55.096,56.189,57.231,58.192,59.042,59.770,60.373,60.858,61.241,61.547
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,62.279,63.298,64.187,64.911,65.461,65.848,66.108,66.302,66.485,66.687,66.933,67.235,67.580,67.951,68.341,68.734,69.108,69.447,69.741,69.990,70.207,70.416,70.635,70.876,71.134,71.388,71.605,71.760,71.843,71.860,71.836,71.803,71.802,71.860,71.992,72.205,72.495,72.838,73.208,73.588,73.955,74.286,74.575,74.820,75.028,75.217,75.418,75.656,75.943,76.281,76.652,77.031,77.389,77.702,77.963,78.174,78.345
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Transform/melt the data so that the years are listed in a single column instead of separate columns.

In [40]:
ids = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
melted_fields = list(life_df.columns.drop(ids))

In [41]:
melted = pd.melt(life_df, id_vars=ids, value_vars=melted_fields, var_name='Year', value_name='Value')

melted.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,65.662
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,32.292
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,33.251
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,62.279
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,NaN


# Practice address missing values for countries using the different approaches (imputation, interpolation, and deletion).

In [42]:
melted.isnull().sum()

Country Name         0
Country Code         0
Indicator Name       0
Indicator Code       0
Year                 0
Value             1301
dtype: int64

In [44]:
#imputation with median value
impute = melted.copy()
impute['Value'] = impute['Value'].fillna(impute['Value'].mean())
impute

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,65.662000
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,32.292000
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,33.251000
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,62.279000
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,63.544406
...,...,...,...,...,...,...
15043,Kosovo,XKX,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,71.646341
15044,"Yemen, Rep.",YEM,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,64.953000
15045,South Africa,ZAF,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,62.774000
15046,Zambia,ZMB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,61.874000


In [46]:
#interpolation with forward fill
ff = melted.copy()
ff['Value'] = ff['Value'].fillna(method='ffill')
ff

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,65.662000
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,32.292000
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,33.251000
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,62.279000
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,62.279000
...,...,...,...,...,...,...
15043,Kosovo,XKX,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,71.646341
15044,"Yemen, Rep.",YEM,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,64.953000
15045,South Africa,ZAF,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,62.774000
15046,Zambia,ZMB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,61.874000


In [52]:
#deletion
delete = melted.copy()
delete = delete.dropna()
delete

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,65.662000
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,32.292000
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,33.251000
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,62.279000
5,Arab World,ARB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,46.825065
...,...,...,...,...,...,...
15043,Kosovo,XKX,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,71.646341
15044,"Yemen, Rep.",YEM,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,64.953000
15045,South Africa,ZAF,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,62.774000
15046,Zambia,ZMB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2016,61.874000
